# Content Check List

1. Find tickers of current S&P500 companies - Completed
2. From S&P 500 companies, select 5 top companies from each industry sector, based on companys’ market capitalization -- Completed
3. Then rank these 5 top company in their own industry sector respectively based on their most important financial metrics -- Almost Completed (details below)

3.1 Ranks Included

'Financials': Return on Equity (ROE)
'Utilities': Debt-To-Equity (D/E) Ratios
'Health Care': Cash, FCF
'Consumer Staples': EBITDA/EV
'Energy': EBITDA/EV
'Industrials': None
'Materials': None
'Communication Services': EPS growth
'Consumer Discretionary': # Sales growth

3.2 Ranks will be included further and the challenges faced

(1). 'Communication Services': P/E ratio 
problems: I haven't get a way to know how to get historical market value per share 

(2) 'Information Technology': Price to Sales 
problems: don't know how to get historical marketcap

(3) 'Real Estate': FFO | NAV 
problems: don't know how to calculate it

### Import Packages

In [4]:
# pip install yfinance
import yfinance as yf
#!pip install yahooquery
from yahooquery import Ticker
import bs4 as bs
import pandas as pd
import pickle
import requests

# Pull Belinda's code-generated files

In [20]:
# ticker_sector_location = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/quant_research/sp500_tickers_updates/"
mkt_cap_location = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/quant_research/moyi/intern_code_projects/belinda_zhang_fin_metrics/"

In [38]:
# df_tickers = pd.read_csv(ticker_sector_location+"sp500.csv")
# df_sectors = pd.read_csv(ticker_sector_location+"sp500_sectors.csv")
df_mkt_cap = pd.read_csv(mkt_cap_location+"info_df.csv")

In [39]:
df_mkt_cap

,TICKER,SECTOR,MARKET_CAPS
0,AAPL,Technology,2127040479232
1,MSFT,Technology,1612060819456
2,AMZN,Consumer Cyclical,1645283377152
3,FB,Communication Services,760663441408
4,GOOGL,Communication Services,1073479942144
...,...,...,...
349,HRB,Consumer Cyclical,2819510784
350,NWS,Communication Services,8592842752
351,UAA,Consumer Cyclical,4198290432
352,UA,Consumer Cyclical,4201654784


In [40]:
df_mkt_cap.columns = ['Ticker', 'Sector', 'MktCap']

In [35]:
df = df_mkt_cap.copy()

### Select top 5 tickers in each sector based on the market cap

In [36]:
def SP500_topN_Mcaps(df, n = 5):
    """
    From a dataframe that contains ticker, market caps and sector, 
    this function Select top N tickers in each sector based on the market cap
    
    Input:
    df: a dataframe that contains 'Ticker', 'MktCap' and 'Sector
    n: top n tickers you want to select
    Output:
    df_top_MktCap: 
    
    """
    df_grouped = df.groupby(['Sector'])
    df_sort= df_grouped.apply(lambda x: x.sort_values(["MktCap"],ascending=False))
    df_sort=df_sort.reset_index(drop=True).rename(columns = {'index':'Ticker'})
    df_top_MktCap = df_sort.groupby('Sector').head(n)
    return df_top_MktCap

In [41]:
df_new = SP500_topN_Mcaps(df, n = 5)
print('length of top tickers:', len(df_new))

length of top tickers: 55


In [42]:
df_new.sort_values(["MktCap"],ascending=False)
df_new.to_excel("top5company.xlsx") 

# Rank these 5 top company in their own industry sector respectively based on their most important financial metrics

## Define the dataframe

In [43]:
# if the previous document is saved as .xlsx in the same folder, import the excel
df_new = pd.read_excel('top5company.xlsx', index_col=0)
# if you have run the module above, the df_new can be displayed by running the code below
df_new.head()

,Ticker,Sector,MktCap
0,LIN,Basic Materials,128345448448
1,APD,Basic Materials,63272878080
2,SHW,Basic Materials,61533204480
3,ECL,Basic Materials,55607267328
4,NEM,Basic Materials,52657364992


In [45]:
df_new

,Ticker,Sector,MktCap
0,LIN,Basic Materials,128345448448
1,APD,Basic Materials,63272878080
2,SHW,Basic Materials,61533204480
3,ECL,Basic Materials,55607267328
4,NEM,Basic Materials,52657364992
17,GOOGL,Communication Services,1073479942144
18,GOOG,Communication Services,1073040392192
19,FB,Communication Services,760663441408
20,DIS,Communication Services,230291734528
21,NFLX,Communication Services,217116098560


In [44]:
# get the unique name of sector
df_new.Sector.unique()

array(['Basic Materials', 'Communication Services', 'Consumer Cyclical',
       'Consumer Defensive', 'Energy', 'Financial Services', 'Healthcare',
       'Industrials', 'Real Estate', 'Technology', 'Utilities'],
      dtype=object)

### Define All the function

In [66]:
def get_ticker(df_new, sector):
    df_sub = list(df_new[df_new['Sector'] == sector].Ticker)
    return df_sub

def get_data_balance_sheet(ticker_set, dataname):
    df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).balance_sheet(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        df_[ticker] = df[ticker]
    return df_

def get_data_cash_flow(ticker_set, dataname):
    df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).cash_flow(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        df_[ticker] = df[ticker]
    return df_

def get_rank(df, SmalltoBig = False):
    df_transposed = df.T
    df_transposed['Rank_last_quarter'] = df_transposed.iloc[:,-1].rank(ascending = SmalltoBig)
    df_transposed = df_transposed.sort_values(by='Rank_last_quarter', ascending= True)
    return df_transposed

def get_sales_growth_each_ticker(ticker_set, dataname):
    # df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).income_statement(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[:-1]
        df.set_index('Date', inplace=True)
        df['sales_growth'] = df.pct_change()
        # df_[ticker] = df[ticker]
        print(df)
    return None

def get_sales_growth_lastQ(ticker_set, dataname):
    df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).income_statement(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[:-1]
        df.set_index('Date', inplace=True)
        df['sales_growth'] = df.pct_change()
        df = df.tail(1).drop(columns = ticker)
        df = df.rename(columns = {'sales_growth':ticker})
        df.reset_index(level=0, inplace=True)
        df.drop(columns = ['Date'])
        df_[ticker] = df[ticker]
    return df_


def get_EBITDA_EV_each_ticker(ticker_set, dataname):
    # df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).income_statement(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[:-1]
        df.set_index('Date', inplace=True)
        df = df.dropna()
        ev = Ticker(ticker).key_stats[ticker]["enterpriseValue"]
        print("{} has enterprice value of {}".format(ticker,ev))
        df['EBITDA_EV'] = df[ticker].div(ev)
        print("{} has historical NormalizedEBITDA and EBITDA/EV:".format(ticker))
        print(df)
        print('------------------------------')     
    return None

def get_EBITDA_EV(ticker_set, dataname):
    df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).income_statement(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[:-1]
        df.set_index('Date', inplace=True)
        df = df.dropna()
        ev = Ticker(ticker).key_stats[ticker]["enterpriseValue"]
        df['EBITDA_EV'] = df[ticker].div(ev)
        df = df.drop(columns = ticker)
        df = df.rename(columns = {'EBITDA_EV':ticker})
        df_[ticker] = df[ticker]
    return df_

def get_EBITDA_EV_lastQ(ticker_set, dataname):
    df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).income_statement(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[:-1]
        df.set_index('Date', inplace=True)
        df = df.dropna()
        ev = Ticker(ticker).key_stats[ticker]["enterpriseValue"]
        df['EBITDA_EV'] = df[ticker].div(ev)
        df = df.tail(1).drop(columns = ticker)
        df = df.rename(columns = {'EBITDA_EV':ticker})
        df.reset_index(level=0, inplace=True)
        df.drop(columns = ['Date'])
        df_[ticker] = df[ticker]
    return df_

def get_data_income_statement_deletelastrow(ticker_set, dataname):
    df_ = pd.DataFrame()
    for ticker in ticker_set: 
        df = Ticker(ticker).income_statement(frequency="q")[['asOfDate', dataname]]
        df = df.rename(columns = {'asOfDate':'Date', dataname:ticker})
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[:-1]
        df.set_index('Date', inplace=True)
        df_[ticker] = df[ticker]
    return df_

# 接下来要开VPN

## 1. Health Care 
### [Cash&Cash Equivalent, FCF]

### Get Cash and Cash Equivalent Data

In [54]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new, 'Healthcare')
    df_cash = get_data_balance_sheet(ticker_set,'CashAndCashEquivalents')
    print("Historical Data for Cash and cash equivalents")
    print(df_cash)
    df_rank = get_rank(df_cash)
    print("----------------------------------------------")
    print("Rank for Cash and cash equivalents")
    print(df_rank)

Historical Data for Cash and cash equivalents
                     JNJ           PFE           ABT           TMO  \
Date                                                                 
2019-06-30  1.437600e+10  1.784000e+09  3.137000e+09  2.288000e+09   
2019-09-30  1.624900e+10  2.785000e+09  4.091000e+09  1.273000e+09   
2019-12-31  1.730500e+10  1.305000e+09  3.860000e+09  2.399000e+09   
2020-03-31  1.553000e+10  2.151000e+09  3.377000e+09  2.981000e+09   
2020-06-30  1.117400e+10  1.801000e+09  4.763000e+09  5.818000e+09   

                    ABBV  
Date                      
2019-06-30  5.172000e+09  
2019-09-30  1.064800e+10  
2019-12-31  3.992400e+10  
2020-03-31  4.114200e+10  
2020-06-30  6.017000e+09  
----------------------------------------------
Rank for Cash and cash equivalents
Date  2019-06-30 00:00:00  2019-09-30 00:00:00  2019-12-31 00:00:00  \
JNJ          1.437600e+10         1.624900e+10         1.730500e+10   
ABBV         5.172000e+09         1.064800e+10    

In [55]:
df_rank

Date,2019-06-30 00:00:00,2019-09-30 00:00:00,2019-12-31 00:00:00,2020-03-31 00:00:00,2020-06-30 00:00:00,Rank_last_quater
JNJ,1.437600e+10,1.624900e+10,1.730500e+10,1.553000e+10,1.117400e+10,1.0
ABBV,5.172000e+09,1.064800e+10,3.992400e+10,4.114200e+10,6.017000e+09,2.0
TMO,2.288000e+09,1.273000e+09,2.399000e+09,2.981000e+09,5.818000e+09,3.0
ABT,3.137000e+09,4.091000e+09,3.860000e+09,3.377000e+09,4.763000e+09,4.0
PFE,1.784000e+09,2.785000e+09,1.305000e+09,2.151000e+09,1.801000e+09,5.0


### Get Free Cash Flow Data

In [56]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new, 'Healthcare')
    df_cash = get_data_cash_flow(ticker_set, 'FreeCashFlow')
    print("Historical Data for Free Cash Flow")
    print(df_cash)
    df_rank = get_rank(df_cash)
    print("----------------------------------------------")
    print("Rank for Free Cash Flow")
    print(df_rank)

ValueError: cannot reindex from a duplicate axis

## 2. Consumer Discretionary （Cyclical）
### [Sales growth]

### Sales Growth

In [62]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new,'Consumer Cyclical')
    print("-------------------------------------")
    print("Historical Total Revenue and Sales Growth Data For 5 Ticker")
    get_sales_growth_each_ticker(ticker_set, 'TotalRevenue')

-------------------------------------
Historical Total Revenue and Sales Growth Data For 5 Ticker
                    AMZN  sales_growth
Date                                  
2019-06-30  6.340400e+10           NaN
2019-09-30  6.998100e+10      0.103732
2019-12-31  8.743600e+10      0.249425
2020-03-31  7.545200e+10     -0.137060
2020-06-30  8.891200e+10      0.178392
                      HD  sales_growth
Date                                  
2019-07-31  3.083900e+10           NaN
2019-10-31  2.722300e+10     -0.117254
2020-01-31  2.578200e+10     -0.052933
2020-04-30  2.826000e+10      0.096114
2020-07-31  3.805300e+10      0.346532
                     NKE  sales_growth
Date                                  
2019-05-31  1.018400e+10           NaN
2019-08-31  1.066000e+10      0.046740
2019-08-31           NaN      0.000000
2019-11-30  1.032600e+10     -0.031332
2020-02-29  1.010400e+10     -0.021499
2020-05-31  6.313000e+09     -0.375198
                     MCD  sales_growth
Date 

In [63]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new,'Consumer Cyclical')
    df_sales_g = get_sales_growth_lastQ(ticker_set, 'TotalRevenue')
    df_rank = get_rank(df_sales_g)
    df_rank = df_rank.rename(columns = {0:'sales_growth'})
    print("-------------------------------------")
    print("Rank for Sales Growth in Last Quarter")
    print(df_rank)

-------------------------------------
Rank for Sales Growth in Last Quarter
      sales_growth  Rank_last_quater
LOW       0.387649               1.0
HD        0.346532               2.0
AMZN      0.178392               3.0
MCD      -0.202125               4.0
NKE      -0.375198               5.0


## 3. Energy
### EBITDA/EV

In [60]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new, 'Energy')
    print('---------- Calculate the EBITDA_EV -------')
    get_EBITDA_EV_each_ticker(ticker_set,'NormalizedEBITDA')
    df = get_EBITDA_EV(ticker_set, 'NormalizedEBITDA')
    print(df)
    df_rank = get_rank(df)
    print("----------------------------------------------")
    print("Rank for EBITDA_EV")
    print(df_rank)

---------- Calculate the EBITDA_EV -------
COP has enterprice value of 47420588032
COP has historical NormalizedEBITDA and EBITDA/EV:
                     COP  EBITDA_EV
Date                               
2019-06-30  3.660000e+09   0.077182
2019-09-30  3.461000e+09   0.072985
2019-12-31  2.665000e+09   0.056199
2020-03-31  5.230000e+08   0.011029
2020-06-30  8.560000e+08   0.018051
------------------------------
KMI has enterprice value of 66028343296
KMI has historical NormalizedEBITDA and EBITDA/EV:
                     KMI  EBITDA_EV
Date                               
2019-06-30  1.697000e+09   0.025701
2019-09-30  1.690000e+09   0.025595
2019-12-31  1.256000e+09   0.019022
2020-03-31  1.741000e+09   0.026367
2020-06-30  2.260000e+09   0.034228
------------------------------
PSX has enterprice value of 41350901760
PSX has historical NormalizedEBITDA and EBITDA/EV:
                     PSX  EBITDA_EV
Date                               
2019-06-30  2.289000e+09   0.055356
2019-09-30

In [61]:
df_rank

Date,2019-06-30 00:00:00,2019-09-30 00:00:00,2019-12-31 00:00:00,2020-03-31 00:00:00,2020-06-30 00:00:00,Rank_last_quater
KMI,0.025701,0.025595,0.019022,0.026367,0.034228,1.0
SLB,0.042061,0.005136,0.080556,-0.012192,0.021068,2.0
COP,0.077182,0.072985,0.056199,0.011029,0.018051,3.0
EOG,0.070371,0.062969,0.069614,0.049610,0.001189,4.0
PSX,0.055356,0.053542,0.036879,0.023700,-0.001548,5.0


## 4. Consumer Staples (Defensive)
### EBITDA/EV

In [65]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new, 'Consumer Defensive')
    print('---------- Calculate the EBITDA_EV -------')
    get_EBITDA_EV_each_ticker(ticker_set,'NormalizedEBITDA')
    print("----------------------------------------------")
    df = get_EBITDA_EV_lastQ(ticker_set,'NormalizedEBITDA')
    print("-------------------------------------")
    print('Show the data for last quarter')
    print(df)
    df_rank = get_rank(df)
    df_rank = df_rank.rename(columns = {0:'EBITDA_EV'})
    print("-------------------------------------")
    print("Rank for EBITDA_EV in Last Quarter")
    print(df_rank)

---------- Calculate the EBITDA_EV -------
WMT has enterprice value of 428617400320
WMT has historical NormalizedEBITDA and EBITDA/EV:
                     WMT  EBITDA_EV
Date                               
2019-07-31  8.276000e+09   0.019309
2019-10-31  7.729000e+09   0.018032
2020-01-31  9.153000e+09   0.021355
2020-04-30  8.779000e+09   0.020482
2020-07-31  1.207500e+10   0.028172
------------------------------
PG has enterprice value of 366104543232
PG has historical NormalizedEBITDA and EBITDA/EV:
                      PG  EBITDA_EV
Date                               
2019-06-30 -4.134000e+09  -0.011292
2019-09-30  5.174000e+09   0.014133
2019-12-31  5.309000e+09   0.014501
2020-03-31  4.397000e+09   0.012010
2020-06-30  4.432000e+09   0.012106
------------------------------
KO has enterprice value of 241097342976
KO has historical NormalizedEBITDA and EBITDA/EV:
                      KO  EBITDA_EV
Date                               
2019-06-30  3.612000e+09   0.014982
2019-09-30 

## 5. Utilities: 
### Debt-To-Equity (D/E) Ratios

In [67]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new, 'Utilities')
    df_equity = get_data_balance_sheet(ticker_set, 'TotalEquityGrossMinorityInterest')
    df_debt = get_data_balance_sheet(ticker_set, 'TotalDebt')
    df_DtoE = df_debt / df_equity
    print(df_equity)
    print(df_debt)
    print(df_DtoE)
    df_rank = get_rank(df_DtoE, SmalltoBig = True)
    print("----------------------------------------------")
    print("Rank for Debt to Equity")
    print(df_rank)

                       D            SO           AEP           SRE  \
Date                                                                 
2019-06-30  2.838100e+10  3.142200e+10  1.942320e+10  1.943400e+10   
2019-09-30  2.995700e+10  3.208200e+10  1.999770e+10  2.055100e+10   
2019-12-31  3.403300e+10  3.205000e+10  1.991320e+10  2.180500e+10   
2020-03-31  3.275200e+10  3.220700e+10  2.000770e+10  2.211500e+10   
2020-06-30  3.088400e+10  3.225200e+10  2.027820e+10  2.538600e+10   

                     EXC  
Date                      
2019-06-30  3.390700e+10  
2019-09-30  3.435200e+10  
2019-12-31  3.457300e+10  
2020-03-31  3.461600e+10  
2020-06-30  3.487100e+10  
                       D            SO           AEP           SRE  \
Date                                                                 
2019-06-30  4.196300e+10  4.615800e+10  2.873520e+10  2.575000e+10   
2019-09-30  4.127300e+10  4.778300e+10  2.942110e+10  2.620600e+10   
2019-12-31  3.789700e+10  4.868600e+10  

In [68]:
df_rank

Date,2019-06-30 00:00:00,2019-09-30 00:00:00,2019-12-31 00:00:00,2020-03-31 00:00:00,2020-06-30 00:00:00,Rank_last_quarter
SRE,1.324997,1.275169,1.183949,1.266968,1.022729,1.0
EXC,1.095172,1.097840,1.093310,1.156257,1.150956,2.0
D,1.478560,1.377741,1.113537,1.212872,1.313172,3.0
SO,1.468971,1.489402,1.519064,1.539852,1.543005,4.0
AEP,1.479427,1.471224,1.533279,1.665729,1.616361,5.0


## 6. Financials (Financial Services): 
### Return on Equity (ROE)

In [69]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new, 'Financial Services')
    df_equity = get_data_balance_sheet(ticker_set, 'StockholdersEquity')
    df_income = get_data_cash_flow(ticker_set, 'NetIncome')
    df_income = df_income[:-1]
    print("----------------------------------------------")
    print("Historical Data of Share Holder Equity")
    print(df_equity)
    print("----------------------------------------------")
    print("Historical Data of Net Income")
    print(df_income)
    df_ItoE = df_income / df_equity
    print("----------------------------------------------")
    print("Historical Data of Return on Equity")
    print(df_ItoE)
    df_rank = get_rank(df_ItoE, SmalltoBig = False)
    print("----------------------------------------------")
    print("Rank for Return on Equity: The higher the better")
    print(df_rank)

ValueError: cannot reindex from a duplicate axis

## 7. Communication Services: 
### P/E ratio | EPS growth

### EPS growth

In [70]:
if __name__ == "__main__":
    ticker_set = get_ticker(df_new, 'Communication Services')
    df = get_data_income_statement_deletelastrow(ticker_set, 'BasicEPS')
    print('Historical Basic EPS data')
    print(df)
    df_change = df.pct_change()
    print("-------------------------------------")
    print('Historical Basic EPS growth data')
    print(df_change)
    print("-------------------------------------")
    print("Rank for Basic EPS growth in Last Quarter: the higher growth rate the better")
    df_rank = get_rank(df_change, SmalltoBig = False)
    print(df_rank)

ValueError: cannot reindex from a duplicate axis